In [1]:
# reqeusts, bs4 import
import requests, bs4
# BeautifulSoup 클래스 import
from bs4 import BeautifulSoup

In [3]:
req_header = { "user-agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36'}

In [4]:
def select_occ1_id(soup):
    occ1_li_tag_list = soup.select("ul#occ1_div li")
    occ1_id_dict = dict()
    for occ1_li_tag in occ1_li_tag_list:
        button_tag = occ1_li_tag.find("button")
        if button_tag:
            occ1_id = button_tag.get("data-item-val")
        span_tag = occ1_li_tag.find("span")
        if occ1_id:
            occ1_id_dict[occ1_id] = span_tag.text
    return occ1_id_dict

def select_occ2_id(soup, id_list):
    occ2_id_dict = dict()
    for id in id_list:
        ul_id = "occ2_ul_" + id
        occ2_li_tag_list = soup.select("ul#" + ul_id + " li")

        for occ2_li_tag in occ2_li_tag_list:
            button_tag = occ2_li_tag.find("button")
            if button_tag:
                occ2_id = tuple(button_tag.get("data-item-val").split("_")) # tuple(occ1_id, occ2_id)
            span_tag = occ2_li_tag.find("span")
            if len(occ2_id) == 2:
                occ2_id_dict[occ2_id[1]] = (occ2_id[0], span_tag.text)
    return occ2_id_dict

In [62]:
import time

def calculate_page_count(count_str):
    count = int(count_str.replace(",", ""))
    page = ((count - 1) // 30) + 1
    return page

def get_page_url_list(occ2_id_dict):
    count_url = "https://job.incruit.com/s_common/searchjob/v3/searchjob_getcount_ajax.asp?occ2="
    url = "https://job.incruit.com/jobdb_list/searchjob.asp?articlecount=30&occ2="

    page_url_list = []
    for occ2_id in occ2_id_dict.keys():
        res = requests.get(count_url + occ2_id)
        if res.ok:
            page_url = url + occ2_id + "&page="
            page_num = calculate_page_count(res.text)
            for page_index in range(1, page_num + 1):
                page_url_list.append(tuple([occ2_id, page_url + str(page_index)]))
        else:
            print(f"에러 코드 = {res.status_code}")
    return page_url_list

def first_cell_parser(li_tag):
    first_cell = li_tag.find('div', class_='cell_first') # 기업 이름과 태그
    company = first_cell.find('a')
    return company.text

def mid_cell_parser(li_tag):
    mid_cell = li_tag.find('div', class_='cell_mid') # 채용 공고
    title = mid_cell.find('a')
    href = title["href"]
    return title.text, href

def get_employ_info(id, link):
    res = requests.get(link, headers=req_header)
    employ_info = dict()
    if res.ok:
        html = res.text
        soup = BeautifulSoup(html, "html.parser")
        ul_tag = soup.find('ul', class_='jc_list')
        info_list = ul_tag.select('li div.txt em')
        employ_info['고용형태'] = info_list[0].text
        employ_info['경력'] = info_list[1].text
        employ_info['근무지역'] = info_list[2].text
        employ_info['학력'] = info_list[3].text
        employ_info['급여조건'] = info_list[4].text
    else:
        print(f"에러 코드 = {res.status_code}")
    return employ_info

def classify_jobs(id1_list, page_url_list):
    total = []
    for id1 in id1_list:
        # id1_dict = dict()
        for id2, page_url in page_url_list: # (occ2_id, page_url)
            res = requests.get(page_url, headers=req_header)
            # id2_dict = dict()
            if res.ok:
                html = res.text
                soup = BeautifulSoup(html, "html.parser")
                li_tag_list = soup.select("li.c_col")
                id2_list = []
                for li_tag in li_tag_list:
                    element = []
                    company_name = first_cell_parser(li_tag)
                    title = mid_cell_parser(li_tag) # (title.text, title_url)
                    employ_info = get_employ_info(id, title[1])
                    element.append(company_name)
                    element.append(title)
                    element.append(employ_info)
                    print(element)
                    # id2_list.append(element)
                    time.sleep(0.2)
                # id2_dict[id2] = id2_list
        # id1_dict[id1] = id2_dict
        # total.append(id1_dict)


In [63]:
url = "https://job.incruit.com/jobdb_list/searchjob.asp?occ1=100&occ1=101&occ1=102&occ1=150&occ1=104&occ1=160&occ1=110&occ1=106&occ1=140&occ1=120&occ1=170&occ1=103&occ1=107&occ1=190&occ1=200&occ1=210&occ1=130"
res = requests.get(url, headers=req_header)

if res.ok:
    html = res.text

    soup = BeautifulSoup(html, "html.parser")
    occ1_id_dict = select_occ1_id(soup)
    # print(occ1_id_dict)
    occ2_id_dict = select_occ2_id(soup, occ1_id_dict.keys())
    # print(occ2_id_dict)
    page_url_list = get_page_url_list(occ2_id_dict)

else:
    print(f"에러 코드 = {res.status_code}")

In [64]:
classify_jobs(occ1_id_dict.keys(), page_url_list)

['인천도시공사', ('2025년도 인천도시공사 신입직원 공개채용 공고', 'https://job.incruit.com/jobdb_info/jobpost.asp?job=2504020000832'), {'고용형태': '정규직 [수습기간: 3개월]', '경력': '신입 ', '근무지역': '인천 > 남동구 외', '학력': '학력무관', '급여조건': '회사내규'}]
['한국석유관리원', ('2025년 한국석유관리원 정규직 채용 공고', 'https://job.incruit.com/jobdb_info/jobpost.asp?job=2504040003430'), {'고용형태': '정규직', '경력': '경력무관 ', '근무지역': '경기 > 성남시 분당구 외', '학력': '학력무관', '급여조건': '회사내규'}]
['한국도심공항(주)', ('한국도심공항 2025년 신입사원 채용공고', 'https://job.incruit.com/jobdb_info/jobpost.asp?job=2504080000971'), {'고용형태': '정규직 [수습기간: 1개월] 외', '경력': '신입 ', '근무지역': '서울 > 강남구', '학력': '대졸↑', '급여조건': '협의'}]
['재단법인 국립자연유산원', ('(재)국립자연유산원 신규직원 채용 공고', 'https://job.incruit.com/jobdb_info/jobpost.asp?job=2504110000751'), {'고용형태': '정규직', '경력': '경력 ', '근무지역': '대전 > 서구', '학력': '고졸↑', '급여조건': '회사내규'}]
['(재)연구개발특구진흥재단', ('2025년 제1차 직원채용 공고', 'https://job.incruit.com/jobdb_info/jobpost.asp?job=2504140000159'), {'고용형태': '정규직 [수습기간: 3개월] 외', '경력': '경력무관 ', '근무지역': '대전 > 유성구, 안산시 단원구 외', '학력': '학력무관', '급여조건': 

KeyboardInterrupt: 